<a href="https://www.kaggle.com/code/mdshahnewazibrahim/ai-news-summarizer-notebook?scriptVersionId=278671772" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as gen
from kaggle_secrets import UserSecretsClient

# 1) Kaggle Secrets থেকে GOOGLE_API_KEY আনছি
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

if not GOOGLE_API_KEY:
    raise RuntimeError("GOOGLE_API_KEY secret set করা নাই – Kaggle Secrets এ add করে নাও।")

# 2) Gemini configure
gen.configure(api_key=GOOGLE_API_KEY)

# 3) **এখানে model hardcode করলাম** – আর auto select নয়
MODEL_NAME = "gemini-2.5-flash"      # lite ব্যবহার করতে চাইলে: "gemini-2.5-flash-lite"
model = gen.GenerativeModel(MODEL_NAME)

print("🤖 Using Gemini model:", MODEL_NAME)


🤖 Using Gemini model: gemini-2.5-flash


In [2]:
def fetch_article_from_url(url: str):
    """Simple scraper: সব <p> ট্যাগ join করে article বানাচ্ছি."""
    resp = requests.get(url, timeout=20)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    title = soup.title.string.strip() if soup.title else "Unknown Title"

    paragraphs = []
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)
        if len(text) > 40:   # খুব ছোট noise বাদ
            paragraphs.append(text)

    article_text = "\n".join(paragraphs)
    return title, article_text


In [3]:
def summarize_with_gemini(article_text: str, url: str = None, language: str = "bn") -> str:
    if not article_text or len(article_text.strip()) < 200:
        return "Article text is too short or could not be extracted."

    # input token কম রাখার জন্য প্রায় ~4000 character এ কেটে নিচ্ছি
    max_chars = 4000
    if len(article_text) > max_chars:
        article_text = article_text[:max_chars]

    if language == "bn":
        instruction = (
            "তুমি একজন AI News Summarizer এজেন্ট। "
            "নিচের নিউজ আর্টিকেলটি পড়ে ৫–৭টি পয়েন্টে সহজ ভাষায় বাংলা সারাংশ লিখো। "
            "মূল ঘটনা, সময়, স্থান, কারণ আর প্রভাব যেন পরিষ্কার থাকে।"
        )
    else:
        instruction = (
            "You are an AI news summarization agent. "
            "Read the article and summarize it in 5–7 clear bullet points in English, "
            "covering who, what, when, where, why, and key outcomes."
        )

    prompt = f"""{instruction}

URL: {url or "N/A"}

Article:
\"\"\"
{article_text}
\"\"\""""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Gemini error: {e}"


In [4]:
def summarize_news_url(url: str, language: str = "bn"):
    print("🌐 Fetching article...")
    title, article_text = fetch_article_from_url(url)
    print("📝 Extracted article length:", len(article_text))

    print("🤖 Sending to Gemini 2.5-flash...")
    summary = summarize_with_gemini(article_text, url=url, language=language)

    return title, summary


In [5]:
url = "https://www.bbc.com/news/articles/c891jp9j79do"  # এখানে যেকোনো news link দাও

title, summary = summarize_news_url(url, language="bn")  # "en" দিলে English summary

print("\n===== 📰 ARTICLE TITLE =====")
print(title)

print("\n===== ✨ GENERATED SUMMARY =====\n")
print(summary)


🌐 Fetching article...
📝 Extracted article length: 5239
🤖 Sending to Gemini 2.5-flash...

===== 📰 ARTICLE TITLE =====
Trump says he will sue BBC for at least $1bn over Panorama edit

===== ✨ GENERATED SUMMARY =====

একজন AI News Summarizer এজেন্ট হিসেবে, বিবিসি নিউজের আর্টিকেলটির সারাংশ নিচে দেওয়া হলো:

**ডোনাল্ড ট্রাম্পের বিবিসিকে মামলা করার হুমকি: মূল ঘটনা, কারণ ও প্রভাব**

১. **মূল ঘটনা ও কারণ:** মার্কিন যুক্তরাষ্ট্রের সাবেক প্রেসিডেন্ট ডোনাল্ড ট্রাম্প বিবিসি (BBC) এর বিরুদ্ধে ১ থেকে ৫ বিলিয়ন ডলার ক্ষতিপূরণ চেয়ে মামলা করার হুমকি দিয়েছেন। কারণ, বিবিসি তাদের 'প্যানোরামা' (Panorama) অনুষ্ঠানে ট্রাম্পের ৬ জানুয়ারী, ২০২১ সালের ভাষণের অংশবিশেষ এমনভাবে সম্পাদনা করেছে, যা সহিংসতার জন্য সরাসরি আহ্বানের ভুল ধারণা তৈরি করেছে বলে ট্রাম্পের অভিযোগ।

২. **বিবিসির প্রতিক্রিয়া:** বিবিসি তাদের ভুল স্বীকার করে ক্ষমা চেয়েছে এবং জানিয়েছে যে ওই সম্পাদিত অংশটি আর সম্প্রচার করা হবে না। তারা স্বীকার করেছে যে তাদের সম্পাদনা 'অনিচ্ছাকৃতভাবে' এমন ধারণা দিয়েছে যে ট্রাম্প সহিংস পদক্ষেপের জন্য সরাসরি আহ্ব

In [6]:
# from urllib.parse import urlparse

# def validate_url(url: str):
#     url = (url or "").strip()
#     if not url:
#         return False, "URL ফাঁকা দেওয়া হয়েছে।"

#     if not (url.startswith("http://") or url.startswith("https://")):
#         return False, "URL অবশ্যই http:// অথবা https:// দিয়ে শুরু হতে হবে।"

#     parsed = urlparse(url)
#     if not parsed.netloc or "." not in parsed.netloc:
#         return False, "URL-এর ডোমেইন অংশটি সঠিক মনে হচ্ছে না।"

#     # চাইলে অতিরিক্ত চেক দিতে পারো (যেমন news site কিনা ইত্যাদি)
#     return True, ""


In [7]:
# def run_news_summarizer(language: str = "bn"):
#     # 1) User input the URL
#     url = input("📝 একটি news URL দিন: ").strip()

#     # 2) Validate the URL
#     is_ok, msg = validate_url(url)
#     if not is_ok:
#         print("❌ Invalid URL:", msg)
#         return

#     # 3) Try summarizing
#     try:
#         title, summary = summarize_news_url(url, language=language)

#         print("\n===== 📰 ARTICLE TITLE =====")
#         print(title)

#         print("\n===== ✨ GENERATED SUMMARY =====\n")
#         print(summary)
#     except Exception as e:
#         print("⚠️ কোনো একটা সমস্যা হয়েছে:", e)

# # এখন শুধু এই লাইনটা চালালেই হবে:
# run_news_summarizer(language="bn")   # English summary চাইলে language="en"
